# PDE Convergence Analysis

This notebook demonstrates the numerical convergence of the PDE engine (Crank-Nicolson) for a European Call option as the grid resolution increases.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qpl.instruments import EuropeanOption
from qpl.market import Market, FlatRateCurve, FlatDividendCurve
from qpl.models import BlackScholesModel
from qpl.engines.analytic import price_european as bs_price, greeks_european as bs_greeks
from qpl.engines.pde.pricers import price_european as pde_price, greeks_european as pde_greeks, PDEConfig


In [ ]:

# 1. Setup
s0 = 100.0
k = 100.0
t = 1.0
r = 0.05
q = 0.02
sigma = 0.2

option = EuropeanOption(strike=k, expiry=t, kind="call")
market = Market(
    spot=s0, 
    rate_curve=FlatRateCurve(r), 
    dividend_curve=FlatDividendCurve(q)
)
model = BlackScholesModel(sigma=sigma)


In [ ]:

# 2. Analytic Baseline
ref_res = bs_price(option, model, market)
ref_greeks = bs_greeks(option, model, market)

print(f"Analytic Reference:")
print(f"  Price: {ref_res.value:.6f}")
print(f"  Delta: {ref_greeks.delta:.6f}")
print(f"  Gamma: {ref_greeks.gamma:.6f}")


In [ ]:

# 3. Convergence Loop
grids = [50, 100, 200, 400, 800]
results = []

print("Running PDE grid metrics...")
for n in grids:
    # Use Equal spatial and temporal steps scaling for simplicity or independent?
    # Here we set n_s = n_t = n
    cfg = PDEConfig(n_s=n, n_t=n, theta=0.5) # Crank-Nicolson
    
    # Compute Price
    p_res = pde_price(option, model, market, cfg=cfg)
    
    # Compute Greeks
    # Note: Greeks in PDE are finite-difference on top of PDE price, so they carry FD error (bump size)
    g_res = pde_greeks(option, model, market, cfg=cfg)
    
    results.append({
        'N': n,
        'Price': p_res.value,
        'Delta': g_res.delta,
        'Gamma': g_res.gamma
    })

df = pd.DataFrame(results)

# Calculate Errors
df['Price_Err'] = (df['Price'] - ref_res.value).abs()
df['Delta_Err'] = (df['Delta'] - ref_greeks.delta).abs()
df['Gamma_Err'] = (df['Gamma'] - ref_greeks.gamma).abs()

print(df[['N', 'Price', 'Price_Err', 'Delta', 'Delta_Err', 'Gamma', 'Gamma_Err']])


In [ ]:

# 4. Visualization
plt.figure(figsize=(12, 5))

# Price Convergence
plt.subplot(1, 2, 1)
plt.loglog(df['N'], df['Price_Err'], 'o-', label='Price Error')
# Compare to O(1/N^2)
plt.loglog(df['N'], 100/df['N']**2, 'k:', label=r'$O(1/N^2)$')
plt.xlabel('Grid Size N (Ns=Nt)')
plt.ylabel('Absolute Error')
plt.title('PDE Price Convergence')
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.legend()

# Greeks Convergence
plt.subplot(1, 2, 2)
plt.loglog(df['N'], df['Delta_Err'], 's-', label='Delta Error')
plt.loglog(df['N'], df['Gamma_Err'], '^-', label='Gamma Error')
plt.xlabel('Grid Size N (Ns=Nt)')
plt.ylabel('Absolute Error')
plt.title('PDE Greeks Convergence')
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.legend()

plt.tight_layout()
plt.show()
